In [26]:
# import modin.pandas as pd
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import sys
sys.path.append('..')
from bigbio_utils import load_dataset_df
from collections import Counter, defaultdict

In [2]:
df = pd.read_csv('/mitchell/entity-linking/mappings/idmapping_selected.tab', delimiter='\t', usecols=[0,1,2], names=['uniprot_ac','uniprot_id','entrez_geneid'], dtype=str).dropna().convert_dtypes().drop_duplicates().reset_index(drop=True)
# df = pd.read_csv('https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/idmapping_selected.tab.gz', delimiter='\t', usecols=[1,2], names=['uniprot_id','entrez_geneid'], dtype=str).dropna().drop_duplicates().reset_index(drop=True)

df.to_feather('../data/mappings/uniprot_entrez.feather')

2024-02-22 18:21:35,516	INFO worker.py:1724 -- Started a local Ray instance.
Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [4]:
df = pd.read_feather('../data/mappings/uniprot_entrez.feather')
df.dtypes

uniprot_ac       string[python]
uniprot_id       string[python]
entrez_geneid    string[python]
dtype: object

In [5]:
df.head()

,uniprot_ac,uniprot_id,entrez_geneid
0,Q6GZX4,001R_FRG3G,2947773
1,Q6GZX3,002L_FRG3G,2947774
2,Q197F8,002R_IIV3,4156251
3,Q197F7,003L_IIV3,4156252
4,Q6GZX2,003R_FRG3G,2947775


In [6]:
entrez_dict = df.set_index('uniprot_ac')['entrez_geneid'].to_dict()

In [22]:
updated_entrez_dict = {'UNIPROT:'+key:'NCBIGENE:'+val for key, val in tqdm(entrez_dict.items())}

  0%|          | 0/13433570 [00:00<?, ?it/s]

100%|██████████| 13433570/13433570 [00:10<00:00, 1260613.83it/s]


In [8]:
dataset = load_dataset_df('sourcedata_nlp')

You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.


In [9]:
dataset

,document_id,offsets,text,type,db_ids,split,mention_id
113,10.1002/eji.200323730,"[[796, 802]]",T cell,[CELL_TYPE],[CL:0000084],train,10.1002/eji.200323730.1
114,10.1002/eji.200323730,"[[809, 816]]",20-4/A4,[CELL_TYPE],[CL:0000084],train,10.1002/eji.200323730.2
115,10.1002/eji.200323730,"[[856, 860]]",NeoR,[GENEPROD],[UNIPROT:Q76MW8],train,10.1002/eji.200323730.3
116,10.1002/eji.200323730,"[[952, 957]]",IFN‐γ,[GENEPROD],[UNIPROT:P01579],train,10.1002/eji.200323730.4
117,10.1002/eji.200323730,"[[991, 998]]",20-4/A4,[CELL_TYPE],[CL:0000084],train,10.1002/eji.200323730.5
...,...,...,...,...,...,...,...
573840,10.7554/elife.04135,"[[4414, 4420]]",SEC31A,[GENEPROD],[UNIPROT:Q3UPL0],train,10.7554/elife.04135.70
573841,10.7554/elife.04135,"[[4533, 4540]]",ERGIC53,[GENEPROD],[UNIPROT:Q9D0F3],train,10.7554/elife.04135.71
573842,10.7554/elife.04135,"[[4595, 4614]]",confocal microscopy,[EXP_ASSAY],[BAO:BAO_0000453],train,10.7554/elife.04135.72
573843,10.7554/elife.04135,"[[4649, 4655]]",SEC31A,[GENEPROD],[UNIPROT:Q3UPL0],train,10.7554/elife.04135.73


In [10]:
geneprod = dataset[dataset.type.map(lambda x: 'GENEPROD' in x)]
geneprod

,document_id,offsets,text,type,db_ids,split,mention_id
115,10.1002/eji.200323730,"[[856, 860]]",NeoR,[GENEPROD],[UNIPROT:Q76MW8],train,10.1002/eji.200323730.3
116,10.1002/eji.200323730,"[[952, 957]]",IFN‐γ,[GENEPROD],[UNIPROT:P01579],train,10.1002/eji.200323730.4
0,10.1002/eji.200323730,"[[1120, 1124]]",NeoR,[GENEPROD],[UNIPROT:Q76MW8],train,10.1002/eji.200323730.6
2,10.1002/eji.200323730,"[[1337, 1341]]",NeoR,[GENEPROD],[UNIPROT:Q76MW8],train,10.1002/eji.200323730.8
3,10.1002/eji.200323730,"[[1399, 1403]]",NeoR,[GENEPROD],[UNIPROT:Q76MW8],train,10.1002/eji.200323730.9
...,...,...,...,...,...,...,...
573838,10.7554/elife.04135,"[[4347, 4351]]",Atg5,[GENEPROD],[NCBIGENE:11793],train,10.7554/elife.04135.68
573840,10.7554/elife.04135,"[[4414, 4420]]",SEC31A,[GENEPROD],[UNIPROT:Q3UPL0],train,10.7554/elife.04135.70
573841,10.7554/elife.04135,"[[4533, 4540]]",ERGIC53,[GENEPROD],[UNIPROT:Q9D0F3],train,10.7554/elife.04135.71
573843,10.7554/elife.04135,"[[4649, 4655]]",SEC31A,[GENEPROD],[UNIPROT:Q3UPL0],train,10.7554/elife.04135.73


In [19]:
entrez_dict['P01579']

'3458'

In [ ]:
updated_entrez_dict['UNIPROT:P01579']

KeyError: 'UNIPROT:P01579'

In [21]:
list(updated_entrez_dict.items())[:10]

[('UNIPROTQ6GZX4', 'NCBIGENE2947773'),
 ('UNIPROTQ6GZX3', 'NCBIGENE2947774'),
 ('UNIPROTQ197F8', 'NCBIGENE4156251'),
 ('UNIPROTQ197F7', 'NCBIGENE4156252'),
 ('UNIPROTQ6GZX2', 'NCBIGENE2947775'),
 ('UNIPROTQ6GZX1', 'NCBIGENE2947776'),
 ('UNIPROTQ197F5', 'NCBIGENE4156254'),
 ('UNIPROTQ6GZX0', 'NCBIGENE2947777'),
 ('UNIPROTQ91G88', 'NCBIGENE1733056'),
 ('UNIPROTQ6GZW9', 'NCBIGENE2947778')]

In [23]:
new_dbids = geneprod['db_ids'].progress_map(lambda x: [updated_entrez_dict[y] if y in updated_entrez_dict else y for y in x])
new_dbids

100%|██████████| 220318/220318 [00:00<00:00, 683410.74it/s]


115       [UNIPROT:Q76MW8]
116        [NCBIGENE:3458]
0         [UNIPROT:Q76MW8]
2         [UNIPROT:Q76MW8]
3         [UNIPROT:Q76MW8]
                ...       
573838    [NCBIGENE:11793]
573840    [NCBIGENE:69162]
573841    [NCBIGENE:70361]
573843    [NCBIGENE:69162]
573844    [NCBIGENE:70361]
Name: db_ids, Length: 220318, dtype: object

In [28]:
Counter([y.split(':')[0] for x in new_dbids for y in x])


Counter({'UNIPROT': 10097, 'NCBIGENE': 240206, 'BAO': 8, 'RFAM': 175})

In [ ]:
entrez_dict = df.set_index('uniprot_id')['entrez_geneid'].to_dict()

In [5]:

df['split_id'] = df['uniprot_id'].progress_map(lambda x: x.split('_')[0])


100%|██████████| 13433570/13433570 [00:11<00:00, 1171118.92it/s]


In [3]:
uniprot_id = 'Q9CPX6'
df.query("uniprot_ac == @uniprot_id")

,uniprot_ac,uniprot_id,entrez_geneid
16088,Q9CPX6,ATG3_MOUSE,67841


In [5]:
split_genes = df['entrez_geneid'].map(lambda x: [y.strip() for y in x.split(';')])
num_geneids = split_genes.map(lambda x: len(x))
num_geneids.max()

269

In [12]:
df['geneids'] = split_genes


,entrez_geneid,geneids
uniprot_id,,
001R_FRG3G,2947773,[2947773]
002L_FRG3G,2947774,[2947774]
002R_IIV3,4156251,[4156251]
003L_IIV3,4156252,[4156252]
003R_FRG3G,2947775,[2947775]
...,...,...
O92813_BLV,2760849,[2760849]
Q64817_AVIS2,4097789,[4097789]
Q9WJ20_9BETR,3416002,[3416002]


In [13]:
df.set_index('uniprot_id')['geneids'].to_dict()

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


{'001R_FRG3G': ['2947773'],
 '002L_FRG3G': ['2947774'],
 '002R_IIV3': ['4156251'],
 '003L_IIV3': ['4156252'],
 '003R_FRG3G': ['2947775'],
 '004R_FRG3G': ['2947776'],
 '005L_IIV3': ['4156254'],
 '005R_FRG3G': ['2947777'],
 '006L_IIV6': ['1733056'],
 '006R_FRG3G': ['2947778'],
 '007R_FRG3G': ['2947779'],
 '007R_IIV3': ['4156256'],
 '008L_IIV3': ['4156257'],
 '009L_FRG3G': ['2947781'],
 '009R_IIV6': ['1733381'],
 '010R_FRG3G': ['2947782'],
 '011L_IIV3': ['4156260'],
 '011R_FRG3G': ['2947783'],
 '012L_FRG3G': ['2947784'],
 '013L_IIV3': ['4156262'],
 '013R_FRG3G': ['2947785'],
 '014R_FRG3G': ['2947786'],
 '015R_FRG3G': ['2947735'],
 '017L_FRG3G': ['2947737'],
 '018L_FRG3G': ['2947738'],
 '019R_FRG3G': ['2947739'],
 '020R_FRG3G': ['2947740'],
 '021L_FRG3G': ['2947741'],
 '022L_IIV3': ['4156271'],
 '023R_FRG3G': ['2947743'],
 '023R_IIV3': ['4156230'],
 '024R_FRG3G': ['2947744'],
 '025R_IIV3': ['4156275'],
 '026R_IIV6': ['1733281'],
 '027R_FRG3G': ['2947747'],
 '028R_FRG3G': ['2947748'],
 '028

In [9]:
split_genes[num_geneids > 3]

4499        [8617663, 8618428, 8618493, 8619412, 8619414, ...
4590                      [115917875, 373190, 373192, 373297]
4715        [3402207, 3403105, 3409480, 3409807, 3411095, ...
8094                [100043684, 100043686, 100043688, 109959]
14746                        [850850, 850852, 850855, 850857]
                                  ...                        
13250156         [108703788, 108703789, 108703792, 108717643]
13257532    [108701579, 108701580, 108701581, 108701582, 3...
13275531    [108695577, 108703793, 108703794, 108703795, 1...
13323192         [106675665, 106676639, 106676640, 106676644]
13354971    [108413812, 108414108, 108414109, 108414295, 1...
Name: entrez_geneid, Length: 2130, dtype: object

In [13]:
converted = df.convert_dtypes()
converted.dtypes

uniprot_id       string
entrez_geneid    object
dtype: object

In [17]:
df['entrez_geneid'].astype(str)

0              2947773
1              2947774
2              4156251
3              4156252
4              2947775
               ...    
250322397    2760849.0
250322480    4097789.0
250322551    3416002.0
250322588    3416005.0
250322693    2746973.0
Name: entrez_geneid, Length: 13433570, dtype: object

In [9]:
subset.shape

(13433570, 3)

In [8]:
df.reset_index(drop=True).to_csv('../data/mappings/uniprot_entrez.csv', index=False)

In [ ]:
pd.to_csv()